# Preprocessing Temperaturdaten ERA 5

Wegen der Datengröße mussten alle Jahre einzeln gedownloaded werden. <br>
Mit diesem Code werden sie zu einem Dataframe zusammengefügt. <br>
Die Rohdaten liegen in NetCDF-Format vor. Diese können nicht direkt aus dem GitHub-Repository geladen werden, <br>
sondern müssen erst lokal gespeichert werden und vom lokalen Laufwerk in das Projekt geladen werden. <br>
Dementsprechend müssen beim Ausführen des Codes die Dateipfade jeweils angepasst werden.

In [1]:
# notwendige Pakete importieren
import xarray as xr
import pandas as pd
import glob
import os

# Daten manuell von GitHub herunterladen und
# lokale working directory setzen
work_dir_tas = r"C:/Users/henry/Documents/A git/projektstudie-murgtal/1_daten/ERA5_temp/raw"
os.chdir(work_dir_tas)

# alle Dateien aufrufen, die mit ".nc" enden, also alle in diesem Ordner
nc_files_tas = sorted(glob.glob(os.path.join(work_dir_tas, "*.nc")))
# Anzahl der aufgerufenen Dateien
print(f"Found {len(nc_files_tas)} NetCDF files.")

Found 43 NetCDF files.


In [3]:
# Dateien aller Jahre zu einer großen Datei zusammenfügen

# alle dataframes listen
all_data_tas = []

# Schleife über alle Dateien/Jahre
for i, years in enumerate(nc_files_tas):
    
    # print(f"Reading year {i+1}/{len(nc_files_tas)}: {os.path.basename(years)}")
    
    # dataset öffnen
    ds_tas = xr.open_dataset(years)
    
    # zu dataframe konvertieren
    df_tas = ds_tas.to_dataframe().reset_index()
    
    # dataframe listen
    all_data_tas.append(df_tas)
    
    # dataframe schließen
    ds_tas.close()

# alle dataframes aneinanderhängen
print("Combining all years...")
combined_df_tas = pd.concat(all_data_tas, ignore_index=True)

#nach Zeit sortieren
combined_df_tas = combined_df_tas.sort_values('valid_time').reset_index(drop=True)

# lokal als csv speichern
output_csv_tas = "C:/Users/henry/Documents/A git/projektstudie-murgtal/1_daten/ERA5_temp/era5_tas_daily_1981_2023.csv"
combined_df_tas.to_csv(output_csv_tas, index=False)

print(f"Saved combined data to: {output_csv_tas}")
print(f"Total rows: {len(combined_df_tas)}")
print(f"Date range: {combined_df_tas['valid_time'].min()} to {combined_df_tas['valid_time'].max()}")

Combining all years...
Saved combined data to: C:/Users/henry/Documents/A git/projektstudie-murgtal/1_daten/ERA5_temp/era5_tas_daily_1981_2023.csv
Total rows: 62820
Date range: 1981-01-01 00:00:00 to 2023-12-31 00:00:00


4 Gitterzellen des ERA-5-Netzes überdecken das Murgtal. <br> Für die folgende Analyse verwenden wir nur einen Wert, den durchschnitt der vier Gitterzellen

In [4]:
# Mittel der 4 Gitterzellen berrechnen

# Liste mit allen Temperaturdaten einladen
input_csv_tas_concat = "C:/Users/henry/Documents/A git/projektstudie-murgtal/1_daten/ERA5_temp/era5_tas_daily_1981_2023.csv"
# als pandas dataframe bearbeiten
df_tas_concat = pd.read_csv(input_csv_tas_concat)


# Datensatz nach Datum (Tag) gruppieren, Mittelwert der 4 Zeilen berrechnen
df_tas_concat_mean = df_tas_concat.groupby('valid_time')['t2m'].mean().reset_index()

# Spalten umbennen
df_tas_concat_mean.columns = ['date', 't2m_mean']

# lokal als .csv speichern
output_csv_tas_concat = "C:/Users/henry/Documents/A git/projektstudie-murgtal/1_daten/ERA5_temp/era5_tas_daily_spatial_mean_1981_2023.csv"
df_tas_concat_mean.to_csv(output_csv_tas_concat, index=False)

print(f"Spatial mean data: {len(df_tas_concat_mean)} rows")
print(f"Saved to: {output_csv_tas_concat}")

Spatial mean data: 15705 rows
Saved to: C:/Users/henry/Documents/A git/projektstudie-murgtal/1_daten/ERA5_temp/era5_tas_daily_spatial_mean_1981_2023.csv


Die Daten liegen in Kelvin vor und müssen in Celcius umgewandelt werden. <br>

In [5]:
# gemittelte Temperaturdaten von lokalem Laufwerk laden
csv_file_units = "C:/Users/henry/Documents/A git/projektstudie-murgtal/1_daten/ERA5_temp/era5_tas_daily_spatial_mean_1981_2023.csv"
# als pandas dataframe berbeiten
df_units = pd.read_csv(csv_file_units)

# umrechnen
df_units['t2m_mean'] = df_units['t2m_mean'] - 273.15

# alte Datei mit neuen Werten überschreiben
df_units.to_csv(csv_file_units, index=False)

print(f"\nUpdated {csv_file_units} with temperatures in Celsius")


Updated C:/Users/henry/Documents/A git/projektstudie-murgtal/1_daten/ERA5_temp/era5_tas_daily_spatial_mean_1981_2023.csv with temperatures in Celsius
